In [56]:
import datetime as dt
import pandas as pd
from random import random, uniform

In [57]:
stations = pd.read_excel("CIMIS Stations List (January20).xlsx")
stations = stations.drop(261)
stations = stations.drop(["DWR Regional Office*", "Name", "Status", "ELEV"], 1)
stations["Connect"] = pd.to_datetime(stations["Connect"]).dt.date
stations.replace("Active", dt.datetime.now(), inplace=True)
stations["Disconnect"] = pd.to_datetime(stations["Disconnect"]).dt.date
stations

C:\Users\profc\AppData\Local\Temp\ipykernel_17052\3340084584.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  stations = stations.drop(["DWR Regional Office*", "Name", "Status", "ELEV"], 1)


,Station Number,County,Latitude,Longitude,Connect,Disconnect
0,1.0,Fresno,36.814000,-119.732000,1982-06-07,1988-09-25
1,2.0,Fresno,36.336222,-120.112906,1982-06-07,2022-11-29
2,3.0,Santa Cruz,36.881000,-121.793000,1982-05-30,1986-08-25
3,4.0,Santa Cruz,36.972000,-121.726000,1982-05-30,1988-04-29
4,5.0,Kern,35.532695,-119.281862,1982-06-01,2022-11-29
...,...,...,...,...,...,...
256,257.0,Kern,35.659128,-117.636925,2018-10-10,2022-11-29
257,258.0,Tulare,36.376917,-119.037972,2018-11-15,2022-11-29
258,259.0,Humboldt,40.604467,-124.243186,2019-08-05,2022-11-29
259,260.0,Siskiyou,41.798331,-122.463425,2019-09-20,2022-11-29


In [58]:
fires = pd.read_csv("fireWithWeather.csv")
for i in range(len(fires.index)):
    fires.loc[i, "Started"] = pd.Timestamp(fires.loc[i, "Started"])
fires

,Fire,Latitude,Longitude,Started,Precipitation30,Precipitation60,Precipitation90,Humidity,Temperature
0,1,37.857000,-120.086000,2013-08-17 15:25:00,0.000000,0.000167,0.001333,39.0,79.8
1,1,34.585595,-118.423176,2013-05-30 15:28:00,0.009333,0.004667,0.010556,39.0,72.8
2,1,33.709500,-116.728850,2013-07-15 13:43:00,0.000000,0.000000,0.000000,23.0,94.0
3,1,33.861570,-116.904270,2013-08-07 14:05:00,0.005333,0.009167,0.010444,39.0,72.4
4,1,40.042630,-121.853970,2013-08-23 14:15:00,0.000333,0.008333,0.006778,58.0,72.5
...,...,...,...,...,...,...,...,...,...
1076,1,39.423833,-121.579738,2019-07-23 14:41:00,NaN,NaN,NaN,NaN,NaN
1077,1,41.946220,-122.401570,2019-06-16 20:33:00,NaN,NaN,NaN,NaN,NaN
1078,1,39.839580,-121.957000,2019-04-30 12:20:00,NaN,NaN,NaN,NaN,NaN
1079,1,33.827979,-117.499619,2019-10-10 12:08:00,NaN,NaN,NaN,NaN,NaN


In [59]:
def genRandoWeather():
    precip30 = 3 * random()
    precip60 = 3 * random()
    precip90 = 3 * random()
    humid = uniform(15, 70)
    temp = uniform(41, 110)
    return (precip30, precip60, precip90, humid, temp)

In [60]:
# Gets a station number given a location and date
# If the location is (0, 0), returns the first station found in that county
# If there is no station active for the given date, returns -1
def getStationNumber(lat, long, day, county):
    
    if not (lat or long):
        for i in range(len(stations.index)):
            if stations.loc[i, "County"] == county and (stations["Connect"][i] <= day and stations["Disconnect"][i] >= day):
                return stations["Station Number"][i]

    minStation = -1
    minDistSquare = -1

    for i in range(len(stations.index)):
        if stations["Connect"][i] <= day and day <= stations["Disconnect"][i]:
            minStation = stations["Station Number"][0]
            initLat = stations.loc[0, "Latitude"]
            initLong = stations.loc[0, "Longitude"]
            minDistanceSquare = (initLat - lat)**2 + (initLong - long)**2
    
    if minStation == -1:
        return -1

    for i in range(1, len(stations.index)):
        statLat = stations.loc[i, "Latitude"]
        statLong = stations.loc[i, "Longitude"]
        if (statLat-lat)**2 + (statLong-long)**2 < minDistanceSquare and stations["Connect"][i] <= day and day <= stations["Disconnect"][i]:
            minStation = stations["Station Number"][i]
            minDistanceSquare = (statLat-lat)**2 + (statLong-long)**2
    
    return minStation

In [61]:
used = []
for i in range(len(fires.index)):
    start = fires.loc[i, "Started"].date()
    lat = fires.loc[i, "Latitude"]
    long = fires.loc[i, "Longitude"]
    stat = getStationNumber(lat, long, start,"")
    used.append((stat, start))
used  

[(148.0, datetime.date(2013, 8, 17)),
 (204.0, datetime.date(2013, 5, 30)),
 (118.0, datetime.date(2013, 7, 15)),
 (238.0, datetime.date(2013, 8, 7)),
 (8.0, datetime.date(2013, 8, 23)),
 (224.0, datetime.date(2013, 9, 9)),
 (8.0, datetime.date(2013, 5, 1)),
 (125.0, datetime.date(2013, 5, 15)),
 (125.0, datetime.date(2013, 5, 15)),
 (106.0, datetime.date(2013, 11, 22)),
 (199.0, datetime.date(2013, 5, 1)),
 (170.0, datetime.date(2013, 9, 8)),
 (84.0, datetime.date(2013, 8, 16)),
 (148.0, datetime.date(2013, 6, 16)),
 (62.0, datetime.date(2013, 8, 5)),
 (125.0, datetime.date(2013, 7, 19)),
 (204.0, datetime.date(2013, 5, 17)),
 (125.0, datetime.date(2013, 7, 23)),
 (238.0, datetime.date(2013, 6, 28)),
 (238.0, datetime.date(2013, 5, 4)),
 (43.0, datetime.date(2013, 7, 3)),
 (170.0, datetime.date(2013, 7, 1)),
 (80.0, datetime.date(2013, 6, 16)),
 (183.0, datetime.date(2013, 2, 24)),
 (77.0, datetime.date(2013, 10, 3)),
 (92.0, datetime.date(2013, 5, 3)),
 (84.0, datetime.date(2013, 10,

In [69]:
for i in range(len(fires.index)):
    (p30, p60, p90, h, t) = genRandoWeather()
    fires.loc[i, "Precipitation30"] = p30
    fires.loc[i, "Precipitation60"] = p60
    fires.loc[i, "Precipitation90"] = p90
    fires.loc[i, "Humidity"] = h
    fires.loc[i, "Temperature"] = t
fires

,Fire,Latitude,Longitude,Started,Precipitation30,Precipitation60,Precipitation90,Humidity,Temperature
0,1,37.857000,-120.086000,2013-08-17 15:25:00,0.141613,0.749929,0.294609,32.491638,107.006948
1,1,34.585595,-118.423176,2013-05-30 15:28:00,0.459999,1.138343,1.232591,65.565533,54.507051
2,1,33.709500,-116.728850,2013-07-15 13:43:00,1.233340,2.327736,2.005262,23.046334,55.787569
3,1,33.861570,-116.904270,2013-08-07 14:05:00,2.546055,0.697333,2.634735,22.412927,98.815980
4,1,40.042630,-121.853970,2013-08-23 14:15:00,0.494467,1.112901,1.393693,66.608484,58.179461
...,...,...,...,...,...,...,...,...,...
1076,1,39.423833,-121.579738,2019-07-23 14:41:00,0.339197,0.667293,0.639993,49.665088,105.833274
1077,1,41.946220,-122.401570,2019-06-16 20:33:00,1.359485,1.064358,0.908651,67.935152,85.014001
1078,1,39.839580,-121.957000,2019-04-30 12:20:00,0.059146,2.399617,1.019838,26.324481,96.770057
1079,1,33.827979,-117.499619,2019-10-10 12:08:00,0.090597,2.667585,1.383824,34.462278,54.315181


In [63]:
begin = min(fires[fires["Started"] != min(fires["Started"])]["Started"])
end = max(fires["Started"])

for i in range(len(stations.index)):
    curr = begin
    while curr < end:
        if (stations[])